In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from alphagogoat.delphox import Delphox
from alphagogoat.utils import DataExtractor
from pathlib import Path
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import json

In [28]:
battles = []
for file in tqdm(list(Path("cache/replays").iterdir())):
    if file.name.endswith(".json"):
        battle_json_path = f"cache/replays/{file.name}"
        battles += DataExtractor(battle_json_path).turns

  0%|          | 0/198 [00:00<?, ?it/s]

2023-05-17 16:21:08,936 - poke-env - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Type: Null', 'Rest', 'p1a: Type: Null', '[from]Sleep Talk'] in battle gen8randombattle-1200357629 turn 14
2023-05-17 16:21:09,577 - poke-env - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Type: Null', 'Crush Claw', 'p2a: Ninetales', '[from]Sleep Talk'] in battle gen8randombattle-1200357629 turn 26
2023-05-17 16:21:09,843 - poke-env - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Type: Null', 'Swords Dance', 'p1a: Type: Null', '[from]Sleep Talk'] in battle gen8randombattle-1200357629 turn 31
2023-05-17 16:21:13,269 - poke-env - WARNING - Rating information regarding an unrecognized username received. Received 'kyuremwhite&gt;blick', while only known players are 'darkell8' and 'kyuremwhite>blick'
